In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

/Users/DanieleP/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [95]:
tf.reset_default_graph()

In [96]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def sample_z(n,m):
    return np.random.uniform(-1.,1., size=[n,m])

In [97]:
# Defining input placeholder and parameters for the generator

Z = tf.placeholder(tf.float32, shape=[None, 100])
with tf.name_scope("generator"):
    G_W1 = tf.Variable(xavier_init([100, 128]),name="G_W1")
    G_b1 = tf.Variable(tf.zeros(shape=[128]),name="G_B1")

    G_W2 = tf.Variable(xavier_init([128, 784]),name="G_W2")
    G_b2 = tf.Variable(tf.zeros(shape=[784]),name="G_B2")

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [98]:
# Defining input placeholder and parameters for the discriminator

X = tf.placeholder(tf.float32, shape=[None, 784])

with tf.name_scope("discriminator"):

    D_W1 = tf.Variable(xavier_init([784, 128]), name="D_W1")
    D_b1 = tf.Variable(tf.zeros(shape=[128]),name="D_B1")

    D_W2 = tf.Variable(xavier_init([128, 1]),name="D_W2")
    D_b2 = tf.Variable(tf.zeros(shape=[1]),name="D_B2")

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [99]:
def generator(z):
    with tf.name_scope("generator"):
        G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
        G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
        G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [100]:
def discriminator(x):
    with tf.name_scope("discriminator"):
        D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
        D_log_prob = tf.matmul(D_h1, D_W2) + D_b2
        D_prob = tf.nn.sigmoid(D_log_prob)
    return D_prob

In [101]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [102]:
G_sample = generator(Z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

In [103]:
D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

In [104]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list = theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list = theta_G)

In [105]:
z_size = 100
mb_size = 16

mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [111]:
sess = tf.Session()
writer = tf.summary.FileWriter('summaries')
writer.add_graph(sess.graph)
sess.run(tf.initialize_all_variables())
merge_summary=tf.summary.merge_all()

In [112]:
if not os.path.exists('../out/'):
    os.makedirs('../out/')

i = 0

for it in range(1000000):
    if it % 100 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_z(16, z_size)})

        fig = plot(samples)
        plt.savefig('../out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_z(mb_size, z_size)})
    
    tf.summary.scalar("D_loss", D_loss_curr)
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_z(mb_size, z_size)})

    if it % 5 == 0:
        s = sess.run(merge_summary, feed_dict={X:X_mb, Z: sample_z(mb_size, z_size)})
        writer.add_summary(s,it)
    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.311
G_loss: 2.735

Iter: 100
D loss: 0.2518
G_loss: 3.141

Iter: 200
D loss: 0.4114
G_loss: 3.873

Iter: 300
D loss: 0.08834
G_loss: 4.298

Iter: 400
D loss: 0.06555
G_loss: 4.744



KeyboardInterrupt: 